# GHCN MODULE

In [2]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
from cputils.mongo import connect_mongo
import logging
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, TimestampType, FloatType
from pyspark.sql.functions import date_format, to_date, to_timestamp
from datetime import datetime

In [9]:
from ghcn_feed import GHCN
#%load_ext autoreload
#%autoreload 2

In [10]:
G = GHCN(downloadDir='/mnt/md1/dat/WeatherData/Feed/GHCN/', # '/data/geoData/wx_ws/ghcn/'
         logfilename='/home/dmasson/temp/ghcn.log',
         experimental_setting=True)
self = G

In [11]:
self.getAllStation_ids()

In [12]:
self.listFiles(first_year=1500)
self.files_df.head()

,file,year
0,1763.csv.gz,1763
1,1764.csv.gz,1764
2,1765.csv.gz,1765
3,1766.csv.gz,1766
4,1767.csv.gz,1767


In [13]:
self.files_df['year'].values.tolist()

[1763,
 1764,
 1765,
 1766,
 1767,
 1768,
 1769,
 1770,
 1771,
 1772,
 1773,
 1774,
 1775,
 1776,
 1777,
 1778,
 1779,
 1780,
 1781,
 1782,
 1783,
 1784,
 1785,
 1786,
 1787,
 1788,
 1789,
 1790,
 1791,
 1792,
 1793,
 1794,
 1795,
 1796,
 1797,
 1798,
 1799,
 1800,
 1801,
 1802,
 1803,
 1804,
 1805,
 1806,
 1807,
 1808,
 1809,
 1810,
 1811,
 1812,
 1813,
 1814,
 1815,
 1816,
 1817,
 1818,
 1819,
 1820,
 1821,
 1822,
 1823,
 1824,
 1825,
 1826,
 1827,
 1828,
 1829,
 1830,
 1831,
 1832,
 1833,
 1834,
 1835,
 1836,
 1837,
 1838,
 1839,
 1840,
 1841,
 1842,
 1843,
 1844,
 1845,
 1846,
 1847,
 1848,
 1849,
 1850,
 1851,
 1852,
 1853,
 1854,
 1855,
 1856,
 1857,
 1858,
 1859,
 1860,
 1861,
 1862,
 1863,
 1864,
 1865,
 1866,
 1867,
 1868,
 1869,
 1870,
 1871,
 1872,
 1873,
 1874,
 1875,
 1876,
 1877,
 1878,
 1879,
 1880,
 1881,
 1882,
 1883,
 1884,
 1885,
 1886,
 1887,
 1888,
 1889,
 1890,
 1891,
 1892,
 1893,
 1894,
 1895,
 1896,
 1897,
 1898,
 1899,
 1900,
 1901,
 1902,
 1903,
 1904,
 1905,

In [ ]:
self.updateThisYear(year=1990)

In [ ]:
from dateutil.relativedelta import relativedelta
today = datetime.today()
to_day = today - relativedelta(months=6)
to_day

In [ ]:
today.year

In [14]:
year=1980
path_to_file00 = '%s%s' % (
    self.downloadDir,
    int(self.files_df.query(
        'year == @year')['file'].values[0])-1)

ValueError: invalid literal for int() with base 10: '1980.csv.gz'

In [17]:
self.files_df.query('year == @year')['file'].values[0][:4]

'1980'

In [18]:
self.files_df.query('year == @year')['file'].values[0]

'1980.csv.gz'

In [23]:
self.df_sta.query('id_station == 67980')

,id_station,mapping1
2972,67980,SPE00119747


In [24]:
from functools import partial

In [ ]:
df0 = self.loadTransform(zipfile='/mnt/md1/dat/WeatherData/Feed/GHCN/1990.csv.gz')

In [ ]:
df0.head()

In [ ]:
df00 = self.loadTransform(zipfile='/mnt/md1/dat/WeatherData/Feed/GHCN/1991.csv.gz')

In [ ]:
# HERE !!!!

df0.shape, df00.shape

In [ ]:

aa = df00.append(df0, ignore_index=True)
aa.head()

In [ ]:
aa.shape

In [ ]:
aa.loc[2, 'date']

In [ ]:
today = datetime.today()
to_day = today - relativedelta(months=6)
to_day

In [ ]:
qq = datetime.date(to_day)

In [ ]:
qq.month

In [ ]:
aa.query('date < @qq')

In [ ]:
to_day

In [7]:
datetime.today().year

2019

In [ ]:
self.df_sta.head()

In [ ]:
dates = df0['date'].unique().tolist()
this_date = dates[3]
this_date

In [ ]:
this_date
df0_day = df0.query('date == @this_date')
df0_day.head()

In [ ]:
df0_day.shape

In [ ]:
var_mapping = {'PRCP': 'col_rain',
               'TMAX': 'col_tmax',
               'TMIN': 'col_tmin',
               'TAVG': 'col_tavg'}
con = self._createMongoConn(self.experimental_setting)

In [ ]:
# Deletion
underlying = 'PRCP'
all_sta = self.df_sta['id_station'].unique().tolist() # 12272
col_dat = con[var_mapping[underlying]]
# modify in delete_many
col_dat.find_one(filter=
#col_dat.delete_many(filter=
                 {'id_station': {'$in': all_sta}, 
                  'Date': datetime(this_date.year, this_date.month, this_date.day, 0, 0)})

In [ ]:
col_dat.count(filter={'id_station': {'$in': all_sta}, 
                      'Date': datetime(this_date.year, this_date.month, this_date.day, 0, 0)})

In [ ]:
# Insertion
df_u = df0_day[['date', 'id_station', underlying]].dropna(subset=[underlying]).reset_index(drop=True).rename(
columns={underlying: 'Measure'}).assign(Date = datetime(this_date.year, this_date.month, this_date.day, 0, 0),
                                        DataQuality = int(5)).drop(columns='date')
df_u['Measure'] = df_u['Measure']/10
df_u.head()

In [ ]:
col_dat.insert_many(df_u.to_dict('records'))

In [ ]:
self.updateThisYear(year=1966)

In [ ]:
year=int(2016)
self.files_df.query('year == @year')['file'].values[0]

In [ ]:
year = 1990
'%s%s' % (self.downloadDir, self.files_df.query('year == @year')['file'].values[0])

In [ ]:
this_file = '%s/1966.csv.gz' % self.downloadDir
this_file

In [ ]:
df = self.loadTransform(zipfile=this_file)

In [ ]:
df.head()

# Spark details

In [ ]:
spark = SparkSession.builder.appName('GHCN_session').getOrCreate()

In [ ]:
DF = spark.read.csv(this_file,
                    header=None)
DF = DF.withColumnRenamed("_c0","station_id")
DF = DF.withColumnRenamed("_c1","date")
DF = DF.withColumnRenamed("_c2","variable")
DF = DF.withColumnRenamed("_c3","value")
DF = DF.withColumnRenamed("_c4","MFLAG")
DF = DF.withColumnRenamed("_c5","QFLAG")
DF = DF.withColumnRenamed("_c6","SFLAG")
DF = DF.withColumnRenamed("_c7","obs_time")
DF = DF.select('station_id', 'date', 'variable','value')
DF.show(5)

In [ ]:
DF = DF.withColumn(colName='station_id', col=DF['station_id'].astype('string'))
DF = DF.withColumn('date', col=to_date('date', 'yyyyMMdd'))
DF = DF.withColumn(colName='variable', col=DF['variable'].astype('string'))
DF = DF.withColumn(colName='value', col=DF['value'].astype('int'))
DF.show(5)

In [ ]:
DF.dtypes

If using directly dataframe:

```python
df = pd.read_csv(this_file, header=None, names=['station_id', 'date', 'variable', 'value', 
                                                'MFLAG', 'QFLAG', 'SFLAG', 'obs_time'])
df.head()

df_sub = df.query('variable in ["PRCP", "TMAX", "TMIN"]')
df_sub.head()

```

In [ ]:
# Focus on PRCP, TMAX, TMIN only
DF_sub = DF[DF['variable'].isin(["PRCP", "TMAX", "TMIN"])]
DF_sub.show(5)

In [ ]:
DF_sub.count()

# Which GHCN stations do we already have ?

In [ ]:
con = self._createMongoConn(experimental_setting=self.experimental_setting)
col_sta = con['col_sta']
col_sta.find_one({'id_provider': 25})

In [ ]:
df_sta = pd.DataFrame(list(col_sta.find({'id_provider': 25, 
                                          'mapping1': 
                                          {'$exists': True}}, 
                                         {'name': 1, 'mapping1': 1,'country': 1, 'id_station': 1, '_id': 0})))
# Post-processing
df_sta['len'] = list(map(lambda x: len(x), df_sta['mapping1']))
df_sta = df_sta.query('len == 11')
df_sta.head()

In [ ]:
df_sta = df_sta[['id_station', 'mapping1']]
df_sta.head()

In [ ]:
ids_ghcn = df_sta['mapping1'].values.tolist()
ids_ghcn[:5]

# Inner joint in Spark

In [ ]:
# Transform pandas dataframe to Spark DataFame
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
DF_sta = sqlContext.createDataFrame(df_sta)
DF_sta.show(5)

In [ ]:
DF_sta.dtypes

In [ ]:
DF_sub2 = DF_sub.join(DF_sta, DF_sub['station_id'] == DF_sta['mapping1'], 'inner').drop('station_id')
DF_sub2.show(5)

In [ ]:
DF_sub2.count() # 4267068

# Pivot Table

In [ ]:
DF_pivot = DF_sub2.groupBy('date', 'id_station').pivot('variable', values=["PRCP", "TMAX", "TMIN"]).avg('value')
DF_pivot.show(5)

In [ ]:
# Create "Average Temperature" column :
DF_pivot = DF_pivot.withColumn('TAVG', 0.5*(DF_pivot['TMAX']+DF_pivot['TMIN']))
DF_pivot.show(5)

In [ ]:
DF_pivot.count()

# Classic MongoDB update

In [ ]:
#df = DF_pivot.toPandas()
df.head()

In [ ]:
con = self._createMongoConn(experimental_setting=self.experimental_setting) # !!!changemeback
col_rain = con['col_rain']
col_tmax = con['col_tmax']
col_tmin = con['col_tmin']
col_tavg = con['col_tavg']

In [ ]:
row = 0
this_obs = df.iloc[row].to_dict()
this_obs

In [ ]:
ind = 200
this_id = 66267
#this_id = int(df.loc[ind, 'id_station']) # 66267
this_date = df.loc[ind, 'date']
#this_date = datetime(2019, 1, 6)
this_date = datetime(this_date.year, 
                     this_date.month, 
                     this_date.day, 0, 0)
update_filter = {'id_station': this_id, 
                 'Date': this_date}
update_filter

In [ ]:
ind = 200
this_id = 66267
#this_id = int(df.loc[ind, 'id_station']) # 66267
this_date = df.loc[ind, 'date']
#this_date = datetime(2019, 1, 6)
this_date = datetime(this_date.year, 
                     this_date.month, 
                     this_date.day, 0, 0)
update_filter = {'id_station': this_id, 
                 'Date': this_date}
update_filter

In [ ]:
var_mapping ={'PRCP': 'col_rain', 'TMAX': 'col_tmax', 'TMIN': 'col_tmin', 'TAVG': 'col_tavg'}
var_mapping

In [ ]:
def updateUnderlying(this_underlying):
    value = df.loc[ind, this_underlying]/10
    update_doc = update_filter.copy()
    update_doc['Measure'] = value
    update_doc['DataQuality'] = int(5)
    con[var_mapping[this_underlying]].update_one(filter=update_filter, 
                                                 update={'$set': update_doc},
                                                 upsert=True)
void = list(map(updateUnderlying, ["PRCP", "TMAX", "TMIN", "TAVG"]))

In [ ]:
con['col_rain'].find_one(filter=update_filter)

In [ ]:
from multiprocessing import Pool

In [ ]:
import os
csv_files = [f for f in os.listdir(self.downloadDir) if f.endswith('.csv.gz')]
files_df = pd.DataFrame({'file': csv_files})
files_df = files_df.assign(
    year= list(map(lambda x: int(x[0:4]), files_df['file']))).sort_values(
    by='year', ascending=True)
firstyear = 1980
files_df = files_df.query('year >= @firstyear').reset_index(drop=True)
files_df.head()

In [ ]:
files_df

# Update line-by-line

In [ ]:
var = ["PRCP", "TMAX", "TMIN"][0]
var